In [1]:
import sys
import json
default = {'training_folder': '../../'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../'}


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import numpy as np
from typing import Literal

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
io_engine = 'pyogrio' if on_lambda else 'pyogrio' #or fiona

In [3]:
base_folder = argv['training_folder']

In [4]:
name = 'Houston_CT_jp' ###
scenario = 'houston'

zones = gpd.read_file(os.path.join(base_folder,'inputs/zones', name+'.geojson'), engine=io_engine)  


In [5]:
zones = zones.set_index('OBJECTID')
zones.index.name = 'index'

In [6]:
zones = zones.to_crs(4326)

In [7]:
pd.set_option('display.max_column', None)
zones.head()

,NAME,ALAND,AWATER,pop_total,pop_5to19,pop_20to54,pop_55up,pop_dens,pop_urban,pop_rural,pop_u_and_r_undefined,pct_urban,pct_rural,emp_total,emp_dens,geometry
index,,,,,,,,,,,,,,,,
1,Census Tract 7601,175464097,2257588,2464,531,1019,914,14.0,0,2464,0,0.0,100.0,383,2.182783,"MULTIPOLYGON (((-96.02022 29.69952, -96.02070 ..."
2,Census Tract 7602.01,72656255,1891572,2078,421,838,819,28.6,651,1427,0,31.3,68.7,612,8.423225,"MULTIPOLYGON (((-96.04924 29.80319, -96.04942 ..."
3,Census Tract 7602.02,78159070,2155333,4450,966,1875,1609,56.9,2408,2042,0,54.1,45.9,938,12.001166,"MULTIPOLYGON (((-96.03308 29.72944, -96.03328 ..."
4,Census Tract 7603.01,104785413,1317743,2773,542,1010,1221,26.5,2068,705,0,74.6,25.4,936,8.932541,"MULTIPOLYGON (((-96.20546 29.88086, -96.20548 ..."
5,Census Tract 7603.02,254780536,3987078,5043,1160,2094,1789,19.8,1258,3785,0,24.9,75.1,2220,8.713381,"MULTIPOLYGON (((-96.31808 29.86316, -96.31877 ..."


In [8]:
tmp_zones = zones[['pop_dens', 'pop_total','emp_total', 'geometry']].rename(columns={'pop_dens': 'density','pop_total':'population', 'emp_total': 'jobs' })

In [9]:
tmp_zones

,density,population,jobs,geometry
index,,,,
1,14.0,2464,383,"MULTIPOLYGON (((-96.02022 29.69952, -96.02070 ..."
2,28.6,2078,612,"MULTIPOLYGON (((-96.04924 29.80319, -96.04942 ..."
3,56.9,4450,938,"MULTIPOLYGON (((-96.03308 29.72944, -96.03328 ..."
4,26.5,2773,936,"MULTIPOLYGON (((-96.20546 29.88086, -96.20548 ..."
5,19.8,5043,2220,"MULTIPOLYGON (((-96.31808 29.86316, -96.31877 ..."
...,...,...,...,...
1607,32.3,4879,1524,"MULTIPOLYGON (((-96.05317 30.10388, -96.05321 ..."
1608,18.2,4337,561,"MULTIPOLYGON (((-95.95036 30.23691, -95.95378 ..."
1609,39.5,2639,313,"MULTIPOLYGON (((-95.80381 30.14372, -95.80427 ..."


In [10]:
output_folder= base_folder +'scenarios/{scen}/inputs/'.format(scen = scenario)
tmp_zones.to_file(output_folder + 'zones.geojson',driver='GeoJSON')

In [11]:
# find meters CRS
from shapely.geometry import Point, LineString
# from quetzal_cyclops
def get_epsg(lat: float, lon: float) -> int:
    '''
    lat, lon or y, x
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

centroid = [*LineString(zones.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])
crs

/tmp/ipykernel_8584/2198738084.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = [*LineString(zones.centroid.values).centroid.coords][0]


32615

In [12]:
zones['area (km2)'] = zones.to_crs(crs).area
zones['area (km2)'].sum()

23286871095.244537